In [2]:
import zipfile
import os
import pandas as pd
from hume import HumeBatchClient
from hume.models.config import FaceConfig

/Users/liuguanhong/Library/Python/3.9/lib/python/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/Users/liuguanhong/Library/Python/3.9/lib/python/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_provider" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/liuguanhong/Library/Python/3.9/lib/python/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_resource" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
# 初始化Hume.ai客户端
client = HumeBatchClient("9FfD6nMezmjb1hvTaWwy09YGBCWDwL9NuK7cFipfZjq49iW7")

# 设置本地图像的路径
image_dir = "./train"
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# 限制只分析前10个图像
image_files = image_files[:15]

# 创建一个空的DataFrame来存储结果
results_df = pd.DataFrame()

# 提交分析任务
config = FaceConfig()  # 初始化分析配置
job = client.submit_job(None, [config], files=image_files)
print("Running facial analysis...")
details = job.await_complete()

# 下载分析结果
artifacts_path = os.path.join(image_dir, 'artifacts.zip')
job.download_artifacts(artifacts_path)

# 解压分析结果
with zipfile.ZipFile(artifacts_path, 'r') as zip_file:
    zip_file.extractall(os.path.join(image_dir, 'predictions'))

# 设置解压目录路径
predictions_path = os.path.join(image_dir, 'predictions')

# 创建一个空的DataFrame来存储结果
results_df = pd.DataFrame()

# 递归遍历predictions目录，找到所有CSV文件
for root, dirs, files in os.walk(predictions_path):
    for file in files:
        if file.endswith('.csv'):
            csv_path = os.path.join(root, file)
            batch_results = pd.read_csv(csv_path)
            results_df = pd.concat([results_df, batch_results], ignore_index=True)

# 保存整合后的结果到CSV
results_csv_path = '/Users/liuguanhong/Desktop/facial_analysis_results.csv'
results_df.to_csv(results_csv_path, index=False)
print(f"Facial analysis results saved to '{results_csv_path}'.")



2024-05-13 12:17:34,392 - INFO - HTTP Request: POST https://api.hume.ai/v0/batch/jobs "HTTP/2 200 OK"
2024-05-13 12:17:34,397 - INFO - Retry attempt 1, waited 0s total
2024-05-13 12:17:34,493 - INFO - HTTP Request: GET https://api.hume.ai/v0/batch/jobs/4465d129-c3b6-4f2a-90fb-47f9d0429fea "HTTP/2 200 OK"
2024-05-13 12:17:34,494 - INFO - Retry iteration 1 failed: 


Running facial analysis...


2024-05-13 12:17:35,495 - INFO - Retry attempt 2, waited 1s total
2024-05-13 12:17:35,548 - INFO - HTTP Request: GET https://api.hume.ai/v0/batch/jobs/4465d129-c3b6-4f2a-90fb-47f9d0429fea "HTTP/2 200 OK"
2024-05-13 12:17:35,549 - INFO - Retry iteration 2 failed: 
2024-05-13 12:17:37,554 - INFO - Retry attempt 3, waited 3s total
2024-05-13 12:17:37,612 - INFO - HTTP Request: GET https://api.hume.ai/v0/batch/jobs/4465d129-c3b6-4f2a-90fb-47f9d0429fea "HTTP/2 200 OK"
2024-05-13 12:17:37,614 - INFO - Retry iteration 3 failed: 
2024-05-13 12:17:41,619 - INFO - Retry attempt 4, waited 7s total
2024-05-13 12:17:41,668 - INFO - HTTP Request: GET https://api.hume.ai/v0/batch/jobs/4465d129-c3b6-4f2a-90fb-47f9d0429fea "HTTP/2 200 OK"
2024-05-13 12:17:41,781 - INFO - HTTP Request: GET https://api.hume.ai/v0/batch/jobs/4465d129-c3b6-4f2a-90fb-47f9d0429fea/artifacts "HTTP/2 200 OK"


Facial analysis results saved to '/Users/liuguanhong/Desktop/facial_analysis_results.csv'.


In [2]:
# 初始化Hume.ai客户端
client = HumeBatchClient("9FfD6nMezmjb1hvTaWwy09YGBCWDwL9NuK7cFipfZjq49iW7")

# 设置本地图像的路径
image_dir = "/Users/liuguanhong/Desktop/train"
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# 创建一个空的DataFrame来存储结果
results_df = pd.DataFrame()

# 分批处理图像以避免内存问题
batch_size = 10  # 你可以根据你的系统资源调整这个大小
for i in range(0, len(image_files), batch_size):
    batch_files = image_files[i:i+batch_size]

    # 提交分析任务
    config = FaceConfig()  # 初始化分析配置
    job = client.submit_job(None, [config], files=batch_files)
    print("Running facial analysis...")
    details = job.await_complete()

    # 下载分析结果
    artifacts_path = os.path.join(image_dir, f'artifacts_batch_{i//batch_size}.zip')
    job.download_artifacts(artifacts_path)

    # 解压分析结果
    with zipfile.ZipFile(artifacts_path, 'r') as zip_file:
        zip_file.extractall(os.path.join(image_dir, f'predictions_batch_{i//batch_size}'))

    # 加载分析结果为DataFrame
    predictions_path = os.path.join(image_dir, f'predictions_batch_{i//batch_size}')
    for csv_file in os.listdir(predictions_path):
        if csv_file.endswith('.csv'):
            batch_results = pd.read_csv(os.path.join(predictions_path, csv_file))
            results_df = pd.concat([results_df, batch_results], ignore_index=True)

# 保存整合后的结果到CSV
results_df.to_csv('/Users/liuguanhong/Desktop/facial_analysis_results.csv', index=False)
print("Facial analysis results saved to 'facial_analysis_results.csv'.")

Running facial analysis...
Running facial analysis...
Running facial analysis...
Running facial analysis...
Running facial analysis...


KeyboardInterrupt: 